In [1]:
import geopy.distance

from main import itinerary, request, scheduler
from main.database import Database
from main.insertion import Insertion

VERBOSE = 1

In [2]:
def request_from_db(database):
    """
    Creation of Request objects from customer information in the configuration file
    """
    db = database
    customers = db.get_customers()
    requests = []
    for customer in customers:
        customer_id = customer
        passenger_id = customer_id
        attributes = db.get_customer_dic(passenger_id)

        coords = db.get_customer_origin(customer_id)
        origin_id = db.get_stop_id([coords[1], coords[0]])

        coords = db.get_customer_destination(customer_id)
        destination_id = db.get_stop_id([coords[1], coords[0]])

        req = request.Request(db, passenger_id, origin_id, destination_id,
                              attributes.get("origin_time_ini"), attributes.get("origin_time_end"),
                              attributes.get("destination_time_ini"), attributes.get("destination_time_end"),
                              attributes.get("npass"))

        requests.append(req)
        if VERBOSE > 1:
            print("Created request from configuration file:")
            print(req.to_string())
    return requests


def itinerary_from_db(database):
    """
    Creation of initial Itinerary objects from vehicle information in the configuration file.
    Initial itineraries contain as first and last stop the warehouse where the vehicle is stored.

    Initialization of itinerary_insertion_dic, a data structure reflecting the insertions contained in each itinerary.
    """
    db = database
    transports = db.get_transports()
    itineraries = []
    itinerary_insertion_dic = {}
    for transport in transports:
        vehicle_id = transport

        coords = db.get_transport_origin(vehicle_id)
        start_stop_id = db.get_stop_id([coords[1], coords[0]])

        coords = db.get_transport_destination(vehicle_id)
        end_stop_id = db.get_stop_id([coords[1], coords[0]])

        attributes = db.get_transport_dic(vehicle_id)

        I = itinerary.Itinerary(db, vehicle_id, attributes.get("capacity"), start_stop_id, end_stop_id,
                                attributes.get("start_time"), attributes.get("end_time"))
        itineraries.append(I)
        itinerary_insertion_dic[vehicle_id] = []
        if VERBOSE > 1:
            print("Created itinerary from configuration file:")
            print(I.to_string())
            print(I.start_stop.to_string())
            print(I.end_stop.to_string())
    return itineraries, itinerary_insertion_dic

In [3]:
database = Database()

Loading STOPS_FILE from ../data/input/ADCAIJ/stops_in_borders3.json
Loading ROUTES_FILE from ../data/input/ADCAIJ/routes_in_borders3.json
Routes loaded in 82.19657278060913s


In [4]:
# config_file = "150r-2000m-60_900-1_5p+10v-8cap-0_960.json" # GUIMABUS
# config_file = "100r-2000m-60_900-1-5p+8v-8cap-22id-0_960.json" # VALENCIA
config_file = "600r-500m-60_360-1_5p+20v-8cap-0_420.json" # ADCAIJ
database.load_config(config_file)

Loading CONFIG from ../data/configs/ADCAIJ/600r-500m-60_360-1_5p+20v-8cap-0_420.json


In [5]:
# Load itineraries from config file
itineraries, itinerary_insertion_dic = itinerary_from_db(database)

# Load requests from config file
requests = request_from_db(database)

# Create and initialize scheduler object
sche = scheduler.Scheduler(database)
sche.pending_requests = requests
sche.itineraries = itineraries
sche.itinerary_insertion_dic = itinerary_insertion_dic

# Schedule all requests by order of issuance
sche.schedule_all_requests_by_time_order(verbose=1)
output = sche.simulation_stats()


Printing all itineraries:

Vehicle with ID veh_01_925113110225 has 52 stops scheduled
	Departure from stop 925113110225 at time 63.96
	--> stop 5250500102, npass 0 -> 1, request_002_1, [65.00, 66.00] (arr, dep), 1.00 min, 0.00 min, customers waited 0.00 min 
	--> stop 5252040204, npass 1 -> 0, request_002_1, [67.76, 68.76] (arr, dep), 1.00 min, 0.00 min
	--> stop 5252010214, npass 0 -> 1, request_001_1, [75.66, 76.66] (arr, dep), 1.00 min, 0.00 min, customers waited 10.66 min 
	--> stop 5252020202, npass 1 -> 3, request_003_2, [83.79, 85.79] (arr, dep), 2.00 min, 0.00 min, customers waited 18.79 min 
	--> stop 5403110202, npass 3 -> 4, request_021_1, [88.63, 89.63] (arr, dep), 1.00 min, 0.00 min, customers waited 11.63 min 
	--> stop 5904210202, npass 4 -> 2, request_003_2, [95.17, 97.17] (arr, dep), 2.00 min, 0.00 min
	--> stop 92501650201, npass 2 -> 1, request_021_1, [109.26, 110.26] (arr, dep), 1.00 min, 0.00 min
	--> stop 5251400112, npass 1 -> 0, request_001_1, [114.26, 115.26] 

In [6]:
for i in sche.itineraries:
    print(i.to_string())

Vehicle with ID veh_01_925113110225 has 52 stops scheduled
	Departure from stop 925113110225 at time 63.96
	--> stop 5250500102, npass 0 -> 1, request_002_1, [65.00, 66.00] (arr, dep), 1.00 min, 0.00 min, customers waited 0.00 min 
	--> stop 5252040204, npass 1 -> 0, request_002_1, [67.76, 68.76] (arr, dep), 1.00 min, 0.00 min
	--> stop 5252010214, npass 0 -> 1, request_001_1, [75.66, 76.66] (arr, dep), 1.00 min, 0.00 min, customers waited 10.66 min 
	--> stop 5252020202, npass 1 -> 3, request_003_2, [83.79, 85.79] (arr, dep), 2.00 min, 0.00 min, customers waited 18.79 min 
	--> stop 5403110202, npass 3 -> 4, request_021_1, [88.63, 89.63] (arr, dep), 1.00 min, 0.00 min, customers waited 11.63 min 
	--> stop 5904210202, npass 4 -> 2, request_003_2, [95.17, 97.17] (arr, dep), 2.00 min, 0.00 min
	--> stop 92501650201, npass 2 -> 1, request_021_1, [109.26, 110.26] (arr, dep), 1.00 min, 0.00 min
	--> stop 5251400112, npass 1 -> 0, request_001_1, [114.26, 115.26] (arr, dep), 1.00 min, 0.00 m

In [7]:
# Creation of stop-id to appearance-order num file
db = database
order = 1
id_order_dict = {}
for stop in db.stops_dic['features']:
    id = int(stop['properties']['id'])
    id_order_dict[id] = order
    order += 1


In [8]:
id_order_dict

{5206020116: 1,
 5206020117: 2,
 5206020120: 3,
 5206020126: 4,
 5206020127: 5,
 5403110202: 6,
 5102010203: 7,
 5102010108: 8,
 5102010206: 9,
 5252040135: 10,
 5252040211: 11,
 5252010214: 12,
 925217900217: 13,
 925217900218: 14,
 5252060137: 15,
 5252060208: 16,
 5252060142: 17,
 5252060140: 18,
 5252060207: 19,
 5252060205: 20,
 5403010125: 21,
 5403010204: 22,
 5206020134: 23,
 5102010105: 24,
 5256100110: 25,
 5102010106: 26,
 5106010209: 27,
 5201110121: 28,
 5102020204: 29,
 5256150204: 30,
 5252010107: 31,
 92633201207: 32,
 5104010118: 33,
 5402010115: 34,
 5107510108: 35,
 5107510203: 36,
 5250500117: 37,
 5250500102: 38,
 5201110203: 39,
 5206020204: 40,
 5206020205: 41,
 5206020210: 42,
 5206020211: 43,
 5206020202: 44,
 5206020207: 45,
 5250710126: 46,
 5250500126: 47,
 5250500202: 48,
 5250600117: 49,
 5250600209: 50,
 5250600217: 51,
 5250750108: 52,
 5250750109: 53,
 5250720215: 54,
 5250720216: 55,
 5250720131: 56,
 5250720130: 57,
 5250720220: 58,
 5250070106: 59,
 

# Neighbouring stops

In [9]:
db = database
for stop in db.stops_dic['features']:
    id = stop['id']
    print(f"{id}: {len(db.get_neighbouring_stops_dict(stop_id=id, max_distance_km=1, geodesic=True))}")

# Version for NUMERIC DISTANCE MATRIX
# for i in range(0, len(db.stops_dic['features'])):
#    print(f"{i}: {[x[0] for x in db.neighbouring_stops(stop_id=i, max_distance_km=0.5)]}")

5206020116: 0
5206020117: 0
5206020120: 23
5206020126: 1
5206020127: 15
5403110202: 29
5102010203: 9
5102010108: 13
5102010206: 4
5252040135: 20
5252040211: 19
5252010214: 20
925217900217: 3
925217900218: 3
5252060137: 25
5252060208: 25
5252060142: 26
5252060140: 25
5252060207: 26
5252060205: 25
5403010125: 31
5403010204: 0
5206020134: 13
5102010105: 15
5256100110: 16
5102010106: 13
5106010209: 7
5201110121: 14
5102020204: 12
5256150204: 15
5252010107: 13
92633201207: 16
5104010118: 13
5402010115: 13
5107510108: 13
5107510203: 14
5250500117: 19
5250500102: 21
5201110203: 9
5206020204: 16
5206020205: 1
5206020210: 21
5206020211: 20
5206020202: 20
5206020207: 14
5250710126: 22
5250500126: 10
5250500202: 11
5250600117: 22
5250600209: 23
5250600217: 11
5250750108: 12
5250750109: 15
5250720215: 1
5250720216: 3
5250720131: 7
5250720130: 11
5250720220: 11
5250070106: 16
5250070216: 21
5250070217: 21
5250070218: 21
5250600118: 21
5250600207: 23
5250600208: 25
5250610123: 24
5250610128: 12
5250

# Stop modification creation

## Itinerary Leg evaluation

In [10]:
from main.scheduler import new_itinerary_from_itinerary
I = new_itinerary_from_itinerary(sche.itineraries[0])
itinerary_id = I.vehicle_id
# Cost of an itinerary (sum of all leg times)
print(f"{I.vehicle_id}, {I.cost}")
for S in I.stop_list:
    # Cost of an itinerary leg (going from S to S.snext)
    try:
        print(f"\t{int(S.id):2d} --> {int(S.snext.id):2d}, {S.leg_time:3.2f}min, {db.get_route_distance_km(S.id, S.snext.id):3.2f}km, {S.passenger_id}")
    except AttributeError:
        print(f"\t{int(S.id):2d} --> None, {S.leg_time:3.2f}, 0km, {S.passenger_id}")

veh_01_925113110225, 208.55880000000002
	925113110225 --> 5250500102, 1.04min, 0.61km, None
	5250500102 --> 5252040204, 1.76min, 1.44km, None
	5252040204 --> 5252010214, 6.90min, 4.57km, None
	5252010214 --> 5252020202, 7.13min, 4.59km, None
	5252020202 --> 5403110202, 2.84min, 1.33km, None
	5403110202 --> 5904210202, 5.54min, 3.99km, None
	5904210202 --> 92501650201, 12.08min, 9.51km, None
	92501650201 --> 5251400112, 4.01min, 2.80km, None
	5251400112 --> 5251300111, 5.72min, 3.84km, None
	5251300111 --> 5254020107, 9.54min, 8.75km, None
	5254020107 --> 5250060205, 8.29min, 6.48km, None
	5250060205 --> 5250600110, 6.09min, 4.06km, None
	5250600110 --> 5251300107, 10.57min, 7.97km, None
	5251300107 --> 5251300103, 4.09min, 1.66km, None
	5251300103 --> 5201110204, 6.81min, 3.13km, None
	5201110204 --> 5251400214, 6.62min, 5.19km, None
	5251400214 --> 5250500121, 6.07min, 3.22km, None
	5250500121 --> 5250500118, 2.24min, 1.42km, None
	5250500118 --> 5250500112, 5.09min, 4.03km, None
	525

In [11]:
from main.leg import Leg
def create_legs_from_itinerary(I, database):
    legs = []
    prev = None

    for S in I.stop_list:
        # Get previous leg
        if len(legs) > 0:
            prev = legs[-1]

        # Get S and T
        T = S.snext
        # if S is not the last stop, T is not None
        if T is not None:
            # Leg creation with temporal and distance costs
            l = Leg(itinerary=I.vehicle_id,
                    origin_stop=S,
                    dest_stop=S.snext,
                    passenger_id=S.snext.passenger_id,
                    time_cost=S.leg_time,
                    dist_cost=database.get_route_distance_km(S.id,S.snext.id),
                    prev=prev, next=None)
            # update result
            legs.append(l)
        # S is the last stop
        else:
            l = Leg(itinerary=I.vehicle_id,
                    origin_stop=S,
                    dest_stop=None,
                    passenger_id="DEPOT",
                    time_cost=S.leg_time,
                    dist_cost=0,
                    prev=prev, # prev -> l connection
                    next=None)
            # update result
            legs.append(l)

        # if S is not the first stop, update prev leg
        if prev is not None:
            prev.set_next(l) # prev -> l connection
    count = 0
    for leg in legs:
        print(f"{count:3d}: {leg.__str__()}")
        count += 1
    return legs

legs = create_legs_from_itinerary(I, db)

  0: veh_01_925113110225 :: leg (925113110225, 63.96) --> (5250500102, 65.00), 1.04min, 0.61km, request_002_1
  1: veh_01_925113110225 :: leg (5250500102, 66.00) --> (5252040204, 67.76), 1.76min, 1.44km, request_002_1
  2: veh_01_925113110225 :: leg (5252040204, 68.76) --> (5252010214, 75.66), 6.90min, 4.57km, request_001_1
  3: veh_01_925113110225 :: leg (5252010214, 76.66) --> (5252020202, 83.79), 7.13min, 4.59km, request_003_2
  4: veh_01_925113110225 :: leg (5252020202, 85.79) --> (5403110202, 88.63), 2.84min, 1.33km, request_021_1
  5: veh_01_925113110225 :: leg (5403110202, 89.63) --> (5904210202, 95.17), 5.54min, 3.99km, request_003_2
  6: veh_01_925113110225 :: leg (5904210202, 97.17) --> (92501650201, 109.26), 12.08min, 9.51km, request_021_1
  7: veh_01_925113110225 :: leg (92501650201, 110.26) --> (5251400112, 114.26), 4.01min, 2.80km, request_001_1
  8: veh_01_925113110225 :: leg (5251400112, 115.26) --> (5251300111, 120.98), 5.72min, 3.84km, request_068_2
  9: veh_01_925113

## Itinerary stop-visit cost computation

In [12]:
def compute_stop_cost_dict(legs):
    # Cost of a stop visit: cost of prev -> S + cost of S -> next
    stop_cost_dict = {}
    for i in range(len(I.stop_list)):
        stop = I.stop_list[i]
        leg = legs[i]

        prev_dist_cost = 0
        next_dist_cost = 0
        prev_time_cost = 0
        next_time_cost = 0

        # not first stop
        if leg.prev is not None:
            prev_dist_cost = leg.prev.dist_cost
            prev_time_cost = leg.prev.time_cost

        # not last stop
        if leg.next is not None:
            next_dist_cost = leg.dist_cost
            next_time_cost = leg.time_cost

        stop_cost_dict[i] = { "stop_id": stop.id,
                             "leg": leg,
                             "dist_cost": prev_dist_cost+next_dist_cost,
                             "time_cost": prev_time_cost+next_time_cost}
    return stop_cost_dict

stop_cost_dict = compute_stop_cost_dict(legs)


In [13]:
for key, value in stop_cost_dict.items():
    print(f"{key}: {value}")

0: {'stop_id': 925113110225, 'leg': <main.leg.Leg object at 0x3575c8f10>, 'dist_cost': 0.6092000000000001, 'time_cost': 1.0366666666666666}
1: {'stop_id': 5250500102, 'leg': <main.leg.Leg object at 0x3575c8340>, 'dist_cost': 2.0502000000000002, 'time_cost': 2.8}
2: {'stop_id': 5252040204, 'leg': <main.leg.Leg object at 0x119a71fd0>, 'dist_cost': 6.0138, 'time_cost': 8.663333333333334}
3: {'stop_id': 5252010214, 'leg': <main.leg.Leg object at 0x1079f85b0>, 'dist_cost': 9.1653, 'time_cost': 14.028333333333332}
4: {'stop_id': 5252020202, 'leg': <main.leg.Leg object at 0x119a88910>, 'dist_cost': 5.919700000000001, 'time_cost': 9.968333333333334}
5: {'stop_id': 5403110202, 'leg': <main.leg.Leg object at 0x357573a90>, 'dist_cost': 5.3177, 'time_cost': 8.381666666666668}
6: {'stop_id': 5904210202, 'leg': <main.leg.Leg object at 0x357573970>, 'dist_cost': 13.4997, 'time_cost': 17.625}
7: {'stop_id': 92501650201, 'leg': <main.leg.Leg object at 0x357573d60>, 'dist_cost': 12.309899999999999, 'tim

## Modification generation according to max_distance_km and MINUMUM_COST_SAVE_MINUTES

In [14]:
# Sort stops according to cost
    # Extract from stop_cost_dict a list of (id, cost)
id_cost_list = []
for key, value in stop_cost_dict.items():
    id_cost_list.append((key, value.get('dist_cost')))
id_cost_list.sort(key=lambda x: x[1], reverse=True)
print(id_cost_list)

[(50, 19.0473), (28, 15.259799999999998), (10, 15.234300000000001), (24, 14.4844), (27, 14.339099999999998), (49, 13.892100000000001), (6, 13.4997), (31, 13.0944), (23, 13.0524), (9, 12.5915), (7, 12.309899999999999), (12, 12.033), (44, 11.1255), (51, 10.7376), (48, 10.571299999999999), (11, 10.538), (30, 10.2482), (21, 10.1304), (13, 9.6381), (3, 9.1653), (20, 9.056799999999999), (19, 8.7259), (16, 8.4155), (45, 8.367600000000001), (15, 8.3186), (47, 7.4441999999999995), (41, 7.2581), (40, 7.1583000000000006), (38, 6.808400000000001), (8, 6.6378), (32, 6.3019), (37, 6.2136000000000005), (2, 6.0138), (4, 5.919700000000001), (22, 5.7736), (18, 5.4416), (5, 5.3177), (43, 5.2508), (14, 4.7888), (17, 4.6382), (29, 4.4505), (46, 4.3388), (42, 3.9259999999999997), (39, 3.8116000000000003), (35, 2.6517), (26, 2.4994), (25, 2.3901), (36, 2.3011), (33, 2.1812), (1, 2.0502000000000002), (34, 1.0646), (0, 0.6092000000000001)]


#### Modification generation with leg dist_cost (Distance)


In [ ]:
MINIMUM_COST_SAVE_KM = 0.1 # 100 meters

# Select a costly stop
for i in range(len(id_cost_list)):
    print()
    # Gets dict entry number that corresponds to the next stop with higher cost
    dic_entry = id_cost_list[i][0]
    print(f"Entry nº{dic_entry}")

    # Gets associated stop information from stop_cost_dict (id, leg, dist_cost, time_cost)
    stop_info = stop_cost_dict[dic_entry]
    print(f"Stop costs: ", stop_info)

    # Get leg corresponding to that stop and previous leg
    stop_id = stop_info['stop_id']
    leg = stop_info['leg']
    prev = leg.prev
    print("\tLeg 1:", prev.__str__())
    print("\tLeg 2:", leg.__str__())
    current_cost = stop_info['dist_cost']
    # Modification type: origin/destination (stop) to be modified
    mod_type = None

    # If the first/last leg has not been chosen
    if prev is not None and leg.next is not None:
        # Get customer/request info associated with the stop to be modified
        # Such a customer is leg.prev.passenger_id
        customer_id = leg.prev.passenger_id
        # customer origin stop id
        origin_coords = db.get_customer_origin(leg.prev.passenger_id)
        origin_id = db.get_stop_id([origin_coords[1], origin_coords[0]])
        # customer destination stop id
        destination_coords = db.get_customer_destination(leg.prev.passenger_id)
        destination_id = db.get_stop_id([destination_coords[1], destination_coords[0]])
        print(f"\t\tTrip to modify: Customer {customer_id} trip: {int(origin_id):3d} --> {int(destination_id):3d}")
        # the stop to be modified is the customer's origin
        if origin_id == stop_id:
            mod_type = "origin"
        # the stop to be modified is the customer's destination
        elif destination_id == stop_id:
            mod_type = "destination"
        else:
            print(f"ERROR :: Stop to be modified {stop_id}, origin_id {origin_id}, destination_id {destination_id}")
            exit()

        # Search for alternative stop among neighbours
        alter = False
        neighbours = db.get_neighbouring_stops_dict(stop_id, max_distance_km=1, geodesic=True)
        if len(neighbours) > 0:
            alter = True
            # Sort neighbours according to distance to the stop
            neighbours.sort(key=lambda x: x[1], reverse=False)

        # If there are neighbouring stops
        if alter:
            modifications = []
            for j in range(len(neighbours)):
                # Calculate costs if visiting each neighbour instead
                # Select next neighbour
                neigh_id = neighbours[j][0]
                neigh_distance = neighbours[j][1]

                # # TIME Cost of a stop visit: cost of prev -> S + cost of S -> next
                # new_prev_time_cost = db.get_route_time_min(prev.origin_stop.id, neigh_id)
                # new_next_time_cost = db.get_route_time_min(neigh_id, leg.dest_stop.id)
                # new_time_cost = new_prev_time_cost + new_next_time_cost
                # # Cost saving
                # time_cost_save = current_cost - new_time_cost

                # DISTANCE cost of a stop visit: cost of prev -> S + cost of S -> next
                new_prev_dist_cost = db.get_route_distance_km(prev.origin_stop.id, neigh_id)
                new_next_dist_cost = db.get_route_distance_km(neigh_id, leg.dest_stop.id)
                new_dist_cost = new_prev_dist_cost + new_next_dist_cost
                dist_cost_save = current_cost - new_dist_cost

                # If the cost is improved
                if dist_cost_save > MINIMUM_COST_SAVE_KM: # Distance cost
                    # Store stop modification proposal
                    mod = {
                        "vehicle_id": itinerary_id,
                        "customer_id": customer_id,
                        "mod_type": mod_type,
                        "stop_id": stop_id,
                        "neigh_id": neigh_id,
                        "neigh_distance": neigh_distance,
                        "cost_save": dist_cost_save,
                        # Ratio between saved distance and walking effort of the customers.
                        # if >1 the vehicle saves more distance than the customers have to walk.
                        "walk_to_save_ratio": dist_cost_save/neigh_distance
                    }
                    modifications.append(mod)
            if len(modifications) > 0:
                modifications.sort(key=lambda x: x["cost_save"], reverse=True)
                print(f"\t\t\tRESULT :: {len(modifications)} modifications found")
                print(f"\t\t\t\tMODIFICATIONS :: ")
                for mod_dict in modifications:
                    print(f"\t\t\t\t{mod_dict}")
            else:
                print(f"\t\t\tRESULT :: No modifications for stop {stop_id} with current parameters.")
        else:
            print(f"\t\t\tWARNING :: No neighbours for stop {stop_id}. Skipping.")
    # If first/last leg have been chosen, skip
    else:
        print(f"\t\t\tWARNING :: Candidate stop corresponds to DEPOT which cannot be modified. Skipping.")

In [ ]:
# Comprovar la feasibility de la modificació --> Segurament siga evident. Si la restricció forta són els temps i es disminuex, entrarà sense problemes
    # Representar la nova request
    # Extraure viatge actual
    # Fer els checks de clavar el nou viatge

# Substituir viatge per viatge alternatiu
    # Extraure viatge previ
    # Clavar viatge modificat
    # Actualitzar costos per veure si la millora es nota

#### Modification generation with leg time_cost (Time)

In [ ]:
MINIMUM_COST_SAVE_MINUTES = 1 # 1 second

# Select a costly stop
for i in range(len(id_cost_list)):
    print()
    # Gets dict entry number that corresponds to the next stop with higher cost
    dic_entry = id_cost_list[i][0]
    print(f"Entry nº{dic_entry}")

    # Gets associated stop information from stop_cost_dict (id, leg, dist_cost, time_cost)
    stop_info = stop_cost_dict[dic_entry]
    print(f"Stop costs: ", stop_info)

    # Get leg corresponding to that stop and previous leg
    stop_id = stop_info['stop_id']
    leg = stop_info['leg']
    prev = leg.prev
    print("\tLeg 1:", prev.__str__())
    print("\tLeg 2:", leg.__str__())
    current_cost = stop_info['time_cost']
    # Modification type: origin/destination (stop) to be modified
    mod_type = None

    # If the first/last leg has not been chosen
    if prev is not None and leg.next is not None:
        # Get customer/request info associated with the stop to be modified
        # Such a customer is leg.prev.passenger_id
        customer_id = leg.prev.passenger_id
        # customer origin stop id
        origin_coords = db.get_customer_origin(leg.prev.passenger_id)
        origin_id = db.get_stop_id([origin_coords[1], origin_coords[0]])
        # customer destination stop id
        destination_coords = db.get_customer_destination(leg.prev.passenger_id)
        destination_id = db.get_stop_id([destination_coords[1], destination_coords[0]])
        print(f"\t\tTrip to modify: Customer {customer_id} trip: {int(origin_id):3d} --> {int(destination_id):3d}")
        # the stop to be modified is the customer's origin
        if origin_id == stop_id:
            mod_type = "origin"
        # the stop to be modified is the customer's destination
        elif destination_id == stop_id:
            mod_type = "destination"
        else:
            print(f"ERROR :: Stop to be modified {stop_id}, origin_id {origin_id}, destination_id {destination_id}")
            exit()

        # Search for alternative stop among neighbours
        alter = False
        neighbours = db.get_neighbouring_stops_dict(stop_id, max_distance_km=1, geodesic=True)
        if len(neighbours) > 0:
            alter = True
            # Sort neighbours according to distance to the stop
            neighbours.sort(key=lambda x: x[1], reverse=False)

        # If there are neighbouring stops
        if alter:
            modifications = []
            for j in range(len(neighbours)):
                # Calculate costs if visiting each neighbour instead
                # Select next neighbour
                neigh_id = neighbours[j][0]
                neigh_distance = neighbours[j][1]

                # TIME Cost of a stop visit: cost of prev -> S + cost of S -> next
                new_prev_time_cost = db.get_route_time_min(prev.origin_stop.id, neigh_id)
                new_next_time_cost = db.get_route_time_min(neigh_id, leg.dest_stop.id)
                new_time_cost = new_prev_time_cost + new_next_time_cost
                # Cost saving
                time_cost_save = current_cost - new_time_cost

                # # DISTANCE cost of a stop visit: cost of prev -> S + cost of S -> next
                # new_prev_dist_cost = db.get_route_distance_km(prev.origin_stop.id, neigh_id)
                # new_next_dist_cost = db.get_route_distance_km(neigh_id, leg.dest_stop.id)
                # new_dist_cost = new_prev_dist_cost + new_next_dist_cost
                # dist_cost_save = current_cost - new_dist_cost

                # If the cost is improved
                if time_cost_save > MINIMUM_COST_SAVE_MINUTES: # Time cost
                    # Store stop modification proposal
                    mod = {
                        "vehicle_id": itinerary_id,
                        "customer_id": customer_id,
                        "mod_type": mod_type,
                        "stop_id": stop_id,
                        "neigh_id": neigh_id,
                        "neigh_distance": neigh_distance,
                        "cost_save": time_cost_save,
                        # Ratio between saved minutes and walking effort of the customers.
                        # if >1 the vehicle saves more minutes than km the customers have to walk.
                        "walk_to_save_ratio": time_cost_save/neigh_distance
                    }
                    modifications.append(mod)
            if len(modifications) > 0:
                modifications.sort(key=lambda x: x["cost_save"], reverse=True)
                print(f"\t\t\tRESULT :: {len(modifications)} modifications found")
                print(f"\t\t\t\tMODIFICATIONS :: ")
                for mod_dict in modifications:
                    print(f"\t\t\t\t{mod_dict}")
            else:
                print(f"\t\t\tRESULT :: No modifications for stop {stop_id} with current parameters.")
        else:
            print(f"\t\t\tWARNING :: No neighbours for stop {stop_id}. Skipping.")
    # If first/last leg have been chosen, skip
    else:
        print(f"\t\t\tWARNING :: Candidate stop corresponds to DEPOT which cannot be modified. Skipping.")

# Modification creation script - Distance cost

In [15]:
MINIMUM_COST_SAVE_KM = 0.1 # 100 meters

db = database
itinerary_modification_dict = {}

for itinerary in sche.itineraries:
	itinerary_modification_dict[itinerary.vehicle_id] = None

for I in sche.itineraries:
    # Initialise mod_per_request_dict
    mod_per_request_dict = {}
    for request_id in I.customer_dict.keys():
        mod_per_request_dict[request_id] = []

    # 1. Create leg from itinerary
    itinerary_id = I.vehicle_id
    legs = create_legs_from_itinerary(I, db)
    # 2. Evaluate legs, creating stop_cost_dict
    stop_cost_dict = compute_stop_cost_dict(legs)
    # 3. Obtain id_cost_list to process entries in descending cost order
    id_cost_list = []
    for key, value in stop_cost_dict.items():
        id_cost_list.append((key, value.get('dist_cost')))
    id_cost_list.sort(key=lambda x: x[1], reverse=True)
    # 4. Process all entries, obtaining the modifications for each of them
    for i in range(len(id_cost_list)):
        modification_list = []
        print()
        # Gets dict entry number that corresponds to the next stop with higher cost
        dic_entry = id_cost_list[i][0]
        print(f"Entry nº{dic_entry}")

        # Gets associated stop information from stop_cost_dict (id, leg, dist_cost, time_cost)
        stop_info = stop_cost_dict[dic_entry]
        print(f"Stop costs: ", stop_info)

        # Get leg corresponding to that stop and previous leg
        stop_id = stop_info['stop_id']
        leg = stop_info['leg']
        prev = leg.prev
        print("\tLeg 1:", prev.__str__())
        print("\tLeg 2:", leg.__str__())
        current_cost = stop_info['dist_cost']
        # Modification type: origin/destination (stop) to be modified
        mod_type = None

        # If the first/last leg has not been chosen
        if prev is not None and leg.next is not None:
            # Get customer/request info associated with the stop to be modified
            # Such a customer is leg.prev.passenger_id
            customer_id = leg.prev.passenger_id
            # customer origin stop id
            origin_coords = db.get_customer_origin(leg.prev.passenger_id)
            origin_id = db.get_stop_id([origin_coords[1], origin_coords[0]])
            # customer destination stop id
            destination_coords = db.get_customer_destination(leg.prev.passenger_id)
            destination_id = db.get_stop_id([destination_coords[1], destination_coords[0]])
            print(f"\t\tTrip to modify: Customer {customer_id} trip: {int(origin_id):3d} --> {int(destination_id):3d}")
            # the stop to be modified is the customer's origin
            if origin_id == stop_id:
                mod_type = "origin"
            # the stop to be modified is the customer's destination
            elif destination_id == stop_id:
                mod_type = "destination"
            else:
                print(f"ERROR :: Stop to be modified {stop_id}, origin_id {origin_id}, destination_id {destination_id}")
                exit()

            # Search for alternative stop among neighbours
            alter = False
            neighbours = db.get_neighbouring_stops_dict(stop_id, max_distance_km=1, geodesic=True)
            if len(neighbours) > 0:
                alter = True
                # Sort neighbours according to distance to the stop
                neighbours.sort(key=lambda x: x[1], reverse=False)

            # If there are neighbouring stops
            if alter:
                modifications = []
                for j in range(len(neighbours)):
                    # Calculate costs if visiting each neighbour instead
                    # Select next neighbour
                    neigh_id = neighbours[j][0]
                    neigh_distance = neighbours[j][1]
                    # Avoid modifications that propose changing a customer origin with their destination and vice versa
                    if (mod_type == "origin" and neigh_id == destination_id) or (mod_type == "destination" and neigh_id == origin_id):
                        continue
                    # # TIME Cost of a stop visit: cost of prev -> S + cost of S -> next
                    # new_prev_time_cost = db.get_route_time_min(prev.origin_stop.id, neigh_id)
                    # new_next_time_cost = db.get_route_time_min(neigh_id, leg.dest_stop.id)
                    # new_time_cost = new_prev_time_cost + new_next_time_cost
                    # # Cost saving
                    # time_cost_save = current_cost - new_time_cost

                    # DISTANCE cost of a stop visit: cost of prev -> S + cost of S -> next
                    new_prev_dist_cost = db.get_route_distance_km(prev.origin_stop.id, neigh_id)
                    new_next_dist_cost = db.get_route_distance_km(neigh_id, leg.dest_stop.id)
                    new_dist_cost = new_prev_dist_cost + new_next_dist_cost
                    dist_cost_save = current_cost - new_dist_cost

                    # If the cost is improved
                    if dist_cost_save > MINIMUM_COST_SAVE_KM: # Distance cost
                        # Store stop modification proposal
                        mod = {
                            "vehicle_id": itinerary_id,
                            "customer_id": customer_id,
                            "mod_type": mod_type,
                            "stop_id": stop_id,
                            "neigh_id": neigh_id,
                            "neigh_distance": neigh_distance,
                            "cost_save": dist_cost_save,
                            # Ratio between saved distance and walking effort of the customers.
                            # if >1 the vehicle saves more distance than the customers have to walk.
                            "walk_to_save_ratio": dist_cost_save/neigh_distance
                        }
                        modifications.append(mod)
                if len(modifications) > 0:
                    modifications.sort(key=lambda x: x["cost_save"], reverse=True)
                    # 5. Store modifications in list, sort by cost_save or ratio
                    mod_per_request_dict[customer_id] += modifications
                    print(f"\t\t\tRESULT :: {len(modifications)} modifications found")
                    print(f"\t\t\t\tMODIFICATIONS :: ")
                    for mod_dict in modifications:
                        print(f"\t\t\t\t{mod_dict}")
                else:
                    print(f"\t\t\tRESULT :: No modifications for stop {stop_id} with current parameters.")
            else:
                print(f"\t\t\tWARNING :: No neighbours for stop {stop_id}. Skipping.")
        # If first/last leg have been chosen, skip
        else:
            print(f"\t\t\tWARNING :: Candidate stop corresponds to DEPOT which cannot be modified. Skipping.")
    # end of THIS itinerary for
    # Sort modifications by cost_save or ratio
    for key in mod_per_request_dict.keys():
        mod_per_request_dict[key].sort(key=lambda x: x["cost_save"], reverse=True)
        # modification_list.sort(key=lambda x: x["cost_save"], reverse=True)
    # 6. Store list in dict
    itinerary_modification_dict[itinerary_id] = mod_per_request_dict

  0: veh_01_925113110225 :: leg (925113110225, 63.96) --> (5250500102, 65.00), 1.04min, 0.61km, request_002_1
  1: veh_01_925113110225 :: leg (5250500102, 66.00) --> (5252040204, 67.76), 1.76min, 1.44km, request_002_1
  2: veh_01_925113110225 :: leg (5252040204, 68.76) --> (5252010214, 75.66), 6.90min, 4.57km, request_001_1
  3: veh_01_925113110225 :: leg (5252010214, 76.66) --> (5252020202, 83.79), 7.13min, 4.59km, request_003_2
  4: veh_01_925113110225 :: leg (5252020202, 85.79) --> (5403110202, 88.63), 2.84min, 1.33km, request_021_1
  5: veh_01_925113110225 :: leg (5403110202, 89.63) --> (5904210202, 95.17), 5.54min, 3.99km, request_003_2
  6: veh_01_925113110225 :: leg (5904210202, 97.17) --> (92501650201, 109.26), 12.08min, 9.51km, request_021_1
  7: veh_01_925113110225 :: leg (92501650201, 110.26) --> (5251400112, 114.26), 4.01min, 2.80km, request_001_1
  8: veh_01_925113110225 :: leg (5251400112, 115.26) --> (5251300111, 120.98), 5.72min, 3.84km, request_068_2
  9: veh_01_925113

In [16]:
# Analise results
for key, value in itinerary_modification_dict.items():
    print(f"Itinerary {key} has modifications for {len(value)} requests")
    mod_per_request_dict = value
    for key, value in mod_per_request_dict.items():
        print(f"\tRequest {key} has {len(value)} modifications")
        if len(value) > 0:
            print(f"\t\tBest modification: {value[0]}")
    print()

Itinerary veh_01_925113110225 has modifications for 25 requests
	Request request_002_1 has 21 modifications
		Best modification: {'vehicle_id': 'veh_01_925113110225', 'customer_id': 'request_002_1', 'mod_type': 'origin', 'stop_id': 5250500102, 'neigh_id': 925113110225, 'neigh_distance': 0.38639018274583853, 'cost_save': 0.8944000000000003, 'walk_to_save_ratio': 2.3147586039687833}
	Request request_001_1 has 5 modifications
		Best modification: {'vehicle_id': 'veh_01_925113110225', 'customer_id': 'request_001_1', 'mod_type': 'origin', 'stop_id': 5252010214, 'neigh_id': 5252010212, 'neigh_distance': 0.9464267109242167, 'cost_save': 1.8685, 'walk_to_save_ratio': 1.9742680319909278}
	Request request_003_2 has 32 modifications
		Best modification: {'vehicle_id': 'veh_01_925113110225', 'customer_id': 'request_003_2', 'mod_type': 'origin', 'stop_id': 5252020202, 'neigh_id': 5403110202, 'neigh_distance': 0.28632857573855613, 'cost_save': 1.735500000000001, 'walk_to_save_ratio': 6.0612182892449

In [17]:
# Get best modifications for each request into a single itinerary-list
all_mods_per_itinerary = {}
for vehicle_id, value in itinerary_modification_dict.items():
    all_mods_per_itinerary[vehicle_id] = []
    mod_per_request_dict = value
    for request_id, value in mod_per_request_dict.items():
        if len(value) > 0:
            all_mods_per_itinerary[vehicle_id].append(value[0])

for key in all_mods_per_itinerary.keys():
    all_mods_per_itinerary[key].sort(key=lambda x: x["cost_save"], reverse=True)

# Get best modifications overall
all_mods = []
for vehicle_id in all_mods_per_itinerary.keys():
    all_mods += all_mods_per_itinerary[vehicle_id]

all_mods.sort(key=lambda x: x["cost_save"], reverse=True)
for entry in all_mods:
    print(f"\t{entry['vehicle_id']:20s}, {entry['customer_id']}, {entry['cost_save']:3.2f}km, (customer walks {entry['neigh_distance']*1000:3.2f} meters)")

	veh_07_925611130111 , request_286_1, 4.38km, (customer walks 966.84 meters)
	veh_04_925113110225 , request_039_1, 4.32km, (customer walks 908.78 meters)
	veh_03_925113110225 , request_499_1, 3.43km, (customer walks 950.62 meters)
	veh_09_5403010204   , request_269_3, 3.43km, (customer walks 983.05 meters)
	veh_14_5252010101   , request_587_1, 3.41km, (customer walks 785.65 meters)
	veh_15_5252010101   , request_440_1, 3.40km, (customer walks 995.84 meters)
	veh_02_925113110225 , request_006_1, 3.30km, (customer walks 992.08 meters)
	veh_19_5254010218   , request_272_1, 3.30km, (customer walks 365.44 meters)
	veh_05_925611130111 , request_013_1, 3.19km, (customer walks 920.24 meters)
	veh_09_5403010204   , request_490_2, 3.11km, (customer walks 958.93 meters)
	veh_13_5252010101   , request_242_1, 3.00km, (customer walks 951.47 meters)
	veh_07_925611130111 , request_080_1, 2.98km, (customer walks 702.16 meters)
	veh_03_925113110225 , request_057_3, 2.96km, (customer walks 999.89 meters)

# Implementation of a modification

In [19]:
# Select the modification to implement
mod = all_mods[0]
vehicle_id = mod["vehicle_id"]
passenger_id = mod["customer_id"]
print(f"Modification to apply: {mod}")
itinerary_to_modify = None
for value in sche.itineraries:
    if value.vehicle_id == vehicle_id:
        itinerary_to_modify = value
I = itinerary_to_modify
print(f"Itinerary to modify:\n",I.to_string_simple())

Modification to apply: {'vehicle_id': 'veh_07_925611130111', 'customer_id': 'request_286_1', 'mod_type': 'destination', 'stop_id': 925114100112, 'neigh_id': 5251400114, 'neigh_distance': 0.9668366522597963, 'cost_save': 4.3766, 'walk_to_save_ratio': 4.526721230282832}
Itinerary to modify:
 Itinerary veh_07_925611130111: [925611130111, 5252050117, 5252040136, 925114700201, 5251400122, 92501650202, 5251300109, 5201110118, 5251300103, 5251300109, 925611130108, 925611130107, 925114700111, 5251400116, 5254010221, 5250060117, 5250060117, 92501620103, 5250740213, 5250600109, 5206020126, 5253010213, 925114220110, 5251300103, 5251300108, 925114100112, 5254020225, 5254020114, 5250060205, 5250060123, 92501620208, 5250610202, 925114010111, 925611130107, 5256120119, 5256120203, 5252040210, 5201110203, 5251400112, 925114700201, 5251300105, 5251400212, 925611130106, 925611130111]
	with a cost increment 181.16


### Customer trip extraction

In [20]:
def get_customer_insertion(vehicle_id, passenger_id):
    itinerary_insertions = sche.itinerary_insertion_dic.get(vehicle_id)
    for insertion in itinerary_insertions:
        if insertion.t.passenger_id == passenger_id:
            return insertion

In [21]:
insertion_to_delete = get_customer_insertion(vehicle_id, passenger_id)
print(insertion_to_delete.to_string())

Insert
	Pickup stop 5251300103 in position 23
	Setdown stop 925114100112 in position 25
	of itinerary veh_07_925611130111: [925611130111, 5252050117, 5252040136, 925114700201, 5251400122, 92501650202, 5251300109, 5201110118, 5251300103, 5251300109, 925611130108, 925611130107, 925114700111, 5251400116, 5254010221, 5250060117, 5250060117, 92501620103, 5250740213, 5250600109, 5206020126, 5253010213, 925114220110, 5251300103, 5251300108, 925114100112, 5254020225, 5254020114, 5250060205, 5250060123, 92501620208, 5250610202, 925114010111, 925611130107, 5256120119, 5256120203, 5252040210, 5201110203, 5251400112, 925114700201, 5251300105, 5251400212, 925611130106, 925611130111]
	with a cost increment of 3.02


In [ ]:
# Itinerary previous to insertion removal
print(I.to_string_simple())
print(I.to_string())
print()
for S in I.stop_list:
    print(S.to_string())

In [22]:
# Extract trip
sche.remove_trip(insertion_to_delete)

# Itinerary after trip removal
print(I.to_string_simple())
print(I.to_string())
for S in I.stop_list:
    print(S.to_string())

Itinerary veh_07_925611130111: [925611130111, 5252050117, 5252040136, 925114700201, 5251400122, 92501650202, 5251300109, 5201110118, 5251300103, 5251300109, 925611130108, 925611130107, 925114700111, 5251400116, 5254010221, 5250060117, 5250060117, 92501620103, 5250740213, 5250600109, 5206020126, 5253010213, 925114220110, 5251300108, 5254020225, 5254020114, 5250060205, 5250060123, 92501620208, 5250610202, 925114010111, 925611130107, 5256120119, 5256120203, 5252040210, 5201110203, 5251400112, 925114700201, 5251300105, 5251400212, 925611130106, 925611130111]
	with a cost increment 178.14
Vehicle with ID veh_07_925611130111 has 42 stops scheduled
	Departure from stop 925611130111 at time 60.82
	--> stop 5252050117, npass 0 -> 1, request_019_1, [76.00, 77.00] (arr, dep), 1.00 min, 0.00 min, customers waited 0.00 min 
	--> stop 5252040136, npass 1 -> 0, request_019_1, [80.25, 81.25] (arr, dep), 1.00 min, 0.00 min
	--> stop 925114700201, npass 0 -> 1, request_018_1, [94.52, 95.52] (arr, dep), 

### Modified trip creation

In [25]:
# Get original customer request attributes
attributes = db.get_customer_dic(passenger_id)

# Current origin stop
coords = db.get_customer_origin(passenger_id)
origin_id = db.get_stop_id([coords[1], coords[0]])
# Current destination stop
coords = db.get_customer_destination(passenger_id)
destination_id = db.get_stop_id([coords[1], coords[0]])

# Get modified stop data
if mod.get('mod_type') == 'origin':
    origin_id = mod.get('neigh_id')
    # origin_coords = db.get_stop_coords(origin_id)

elif mod.get('mod_type') == 'destination':
    destination_id = mod.get('neigh_id')
    # destination_coords = db.get_stop_coords(destination_id)

else:
    print(f"ERROR :: Mod type {mod.get('mod_type')} not recognized.")
    exit()


# Update origin/destination stop id

# Initialise request stops
mod_Request = request.Request(db, passenger_id, origin_id, destination_id,
                      attributes.get("origin_time_ini"), attributes.get("origin_time_end"),
                      attributes.get("destination_time_ini"), attributes.get("destination_time_end"),
                      attributes.get("npass"))

# Create insertion with same indexes as original one
index_Spu = insertion_to_delete.index_Spu
index_Ssd = insertion_to_delete.index_Ssd

mod_insertion = Insertion(
    itinerary=I,
    trip=mod_Request,
    index_Spu=index_Spu,
    index_Ssd=index_Ssd,
    cost_increment=-1
)

In [26]:
print(f"Modified insertion: {mod_insertion.to_string()}")

Modified insertion: Insert
	Pickup stop 5251300103 in position 23
	Setdown stop 5251400114 in position 25
	of itinerary veh_07_925611130111: [925611130111, 5252050117, 5252040136, 925114700201, 5251400122, 92501650202, 5251300109, 5201110118, 5251300103, 5251300109, 925611130108, 925611130107, 925114700111, 5251400116, 5254010221, 5250060117, 5250060117, 92501620103, 5250740213, 5250600109, 5206020126, 5253010213, 925114220110, 5251300108, 5254020225, 5254020114, 5250060205, 5250060123, 92501620208, 5250610202, 925114010111, 925611130107, 5256120119, 5256120203, 5252040210, 5201110203, 5251400112, 925114700201, 5251300105, 5251400212, 925611130106, 925611130111]
	with a cost increment of -1.00


### Feasibility of a given modification

In [27]:
from main.scheduler import new_stop_from_stop, new_itinerary_from_itinerary

# Check feasibility of inserting the modified customer trip in the same indexes of the itinerary

# Extract Request's stops
Spu = new_stop_from_stop(mod_Request.Spu)
print(f"Spu is {Spu.to_string_trip()}")
Ssd = new_stop_from_stop(mod_Request.Ssd)
print(f"Ssd is {Ssd.to_string_trip()}")
print()
passenger_id = mod_Request.passenger_id
# Extract itinerary's stop list
filtered_stops_i = I.stop_list

# Feasibility of PICKUP stop
# Index to insert pickup stop
index_stop_i = index_Spu - 1
try:
    R = filtered_stops_i[index_stop_i]
except IndexError:
    print("ERROR Searching inside itinerary {}".format(I.vehicle_id))
    print(I.to_string())
    print()
    print("with the following list of filtered stops: {}".format([x.id for x in filtered_stops_i]))
    for x in filtered_stops_i:
        print(x.to_string())
    print()
    print("and an index_stop_i of: {}".format(index_stop_i))
    print(len(filtered_stops_i), index_stop_i)
    exit()

T = R.snext
print(f"R is {R.to_string()}")
print(f"T is {T.to_string()}")
# Check feasibility of inserting Spu in R's position, so that leg (R -> R.rnext)
# becomes (Spu -> R.snext) therefore creating also a new leg (R -> Spu)
test, code = I.pickup_insertion_feasibility_check(mod_Request, Spu, R, T)
print(f"Spu insertion in position {index_stop_i+1}: {test}\n")
# Feasibility of SETDOWN stop
if test:
    # Once we select a feasible leg to insert Spu, store the index
    index_Spu = index_stop_i + 1
    # Copy of the itinerary to avoid modifications over the original
    I_with_Spu = new_itinerary_from_itinerary(I)
    # I_with_Spu = copy_Itinerary(I)
    # Insert Spu in the itinerary and re-calculate EAT carried forward over its putative successors
    I_with_Spu.insert_stop(Spu, index_Spu)
    # Compute the insertion's net additional cost
    I_with_Spu.compute_cost()
    print("Itinerary after insertion of Spu: {}\n".format(I_with_Spu.to_string_simple()))
    # Filter list of stops to keep only those not yet visited
    filtered_stops_j = [new_stop_from_stop(x) for x in I_with_Spu.stop_list]

    # Index to insert setdown stop
    index_stop_j = index_Ssd - 1
    R = filtered_stops_j[index_stop_j]
    T = R.snext
    print(f"\tR is {R.to_string()}")
    print(f"\tT is {T.to_string()}")
    test, code = I_with_Spu.setdown_insertion_feasibility_check(mod_Request, index_Spu,
                                                                index_stop_j + 1,
                                                                I_with_Spu.stop_list, Ssd, R, T)
    print(f"\tSsd insertion in position {index_stop_j+1}: {test}")
    if test:
        I_with_Spu_Ssd = new_itinerary_from_itinerary(I_with_Spu)
        I_with_Spu_Ssd.insert_stop(Ssd, index_Ssd)
        # Compute the insertion's net additional cost
        I_with_Spu_Ssd.compute_cost()
        print("Itinerary after insertion of Ssd: {}\n".format(I_with_Spu_Ssd.to_string_simple()))

        mod_insertion = Insertion(
                            itinerary=I,
                            trip=mod_Request,
                            index_Spu=index_Spu,
                            index_Ssd=index_Ssd,
                            cost_increment=-1
                        )

Spu is Stop with ID 5251300103 located at (39.51, -0.41)
	 Time window
		 start_time: 220.00
		 latest: 240.00, end_time: 241.00

Ssd is Stop with ID 5251400114 located at (39.51, -0.44)
	 Time window
		 start_time: 226.34
		 latest: 254.85, end_time: 255.85


R is Stop with ID 925114220110 located at (39.50, -0.41)
	 prev stop 5253010213
	 next stop 5251300108, with leg time 5.03
	 Time window
		 start_time: 187.55
		 EAT: 226.35, EAT_F: 226.35
		 LDT: 230.97, LDT_F: 227.88
		 slack: 0.00
		 latest: 226.88, end_time: 227.88
	 Arrival at 226.35, departure at 227.35
	 Capacity
		 Passengers on board on departure from here: 0	 Number of seats reserved on departure from here: 0

T is Stop with ID 5251300108 located at (39.52, -0.42)
	 prev stop 925114220110
	 next stop 5254020225, with leg time 11.38
	 Time window
		 start_time: 216.00
		 EAT: 232.38, EAT_F: 232.38
		 LDT: 242.62, LDT_F: 237.00
		 slack: 0.00
		 latest: 236.00, end_time: 237.00
	 Arrival at 234.16, departure at 235.16
	 C

In [ ]:
print(I.to_string_simple())

### Modified trip insertion

In [28]:
# Apply insertion with Scheduler.insert_trip(modified_insertion)
# Check Itinerary integrity (times)

# Itinerary previous to insertion
print(I.to_string_simple())
print(I.to_string())
print()

# Insert trip
sche.insert_trip(mod_insertion)

# Itinerary after trip insertion
print(I.to_string_simple())
print(I.to_string())

Itinerary veh_07_925611130111: [925611130111, 5252050117, 5252040136, 925114700201, 5251400122, 92501650202, 5251300109, 5201110118, 5251300103, 5251300109, 925611130108, 925611130107, 925114700111, 5251400116, 5254010221, 5250060117, 5250060117, 92501620103, 5250740213, 5250600109, 5206020126, 5253010213, 925114220110, 5251300108, 5254020225, 5254020114, 5250060205, 5250060123, 92501620208, 5250610202, 925114010111, 925611130107, 5256120119, 5256120203, 5252040210, 5201110203, 5251400112, 925114700201, 5251300105, 5251400212, 925611130106, 925611130111]
	with a cost increment 178.14
Vehicle with ID veh_07_925611130111 has 42 stops scheduled
	Departure from stop 925611130111 at time 60.82
	--> stop 5252050117, npass 0 -> 1, request_019_1, [76.00, 77.00] (arr, dep), 1.00 min, 0.00 min, customers waited 0.00 min 
	--> stop 5252040136, npass 1 -> 0, request_019_1, [80.25, 81.25] (arr, dep), 1.00 min, 0.00 min
	--> stop 925114700201, npass 0 -> 1, request_018_1, [94.52, 95.52] (arr, dep), 

# TO DO

In [ ]:
p1 = (-0.4565131701, 39.4824055691)
p2 = (-0.4557867049, 39.4805017556)

p1 = (39.4824055691, -0.4565131701)
p2 = (39.4805017556, -0.4557867049)

In [ ]:
db.get_stop(p1)

In [ ]:
db.get_stop(p2)

In [ ]:
db.get_route(p1, p2)

In [ ]:
geopy.distance.distance(p1, p2).km

### Check Itinerary integrity and cost reduction

# Discarded cells

In [ ]:
# # Sort route legs according to cost
# sorted_legs = sorted(legs, key=lambda leg: leg.cost, reverse=True)
# # Select a costly leg
# leg = sorted_legs[0]
# print(leg.__str__())
# # Check if the leg is for passenger pickup or setdown
# leg_stop_id = leg.dest_stop.id
# # customer origin stop id
# origin_coords = db.get_customer_origin(leg.passenger_id)
# origin_id = db.get_stop_id([origin_coords[1], origin_coords[0]])
# # customer destination stop id
# destination_coords = db.get_customer_destination(leg.passenger_id)
# destination_id = db.get_stop_id([destination_coords[1], destination_coords[0]])
# print(f"Customer trip from {origin_id:3d} --> {destination_id:3d}")
# pickup = True
# if leg.dest_stop.id == origin_id:
#     print(f"\tLeg corresponds to customer pickup")
# else:
#     pickup = False
#     print(f"\tLeg corresponds to customer setdown")
#
# # Look for alternative stops within the neighbours
# if pickup:
#     # Stop to be modified
#     stop_id = origin_id
# else:
#     stop_id = destination_id
#
# neighbours = db.get_neighbouring_stops(stop_id)
# print(neighbours)
#
# # Calculate costs if going to any of the neighbours:
# #   S -> neighbour (can be calculated from current leg)
# #   neighbour -> T (must extract next leg and see the subsequent stop)
# leg_origin_stop_id = leg.origin_stop.id
# for id, distance in neighbours:
#     print(db.get_route_time_min(leg_origin_stop_id, id))
#
# # SEGUIR ACI

In [ ]:
# stop_customer_list = []
# print("stop_id, setdown, pickup")
# for S in I.stop_list:
#     if S.snext is not None:
#         stop_customer_list.append((S.id, S.passenger_id, S.snext.passenger_id))
#     else:
#         stop_customer_list.append((S.id, S.passenger_id, None))

In [ ]:
# stop_customer_list

### Sii apareixen dues parades amb la mateixa id seguides, més d'un client ha sigut servit amb eixa visita. Encara que coste arribar a eixa parada, ja tenim demanda agrupada, així que no es deuria detectar com a una mala ruta.


### Necessite representar, doncs, el benefici associat a cada "leg", encara que siga per mitjà del nombre de clients servits amb cada trajecte


In [ ]:
# # Load stops file
# # Load routes file
# db = Database(config_file)

In [ ]:
# def get_neighbouring_stops(stop_id, max_distance_km=1):
#     distance_matrix = db.get_distance_matrix()
#     neighbours = []
#     for i in range(0, len(distance_matrix[stop_id])):
#         if i != stop_id:
#             if distance_matrix[stop_id][i] <= max_distance_km:
#                 neighbours.append((i, distance_matrix[stop_id][i]))
#     return neighbours

In [ ]:
# Testing geopy.distance
import geopy.distance
s1 = database.stops_dic['features'][0]
s2 = database.stops_dic['features'][1]


p1 = s1['geometry']['coordinates']
p2 = s2['geometry']['coordinates']

id1 = s1['properties']['id']
id2 = s2['properties']['id']
print(id1, id2)

print("Geopy", geopy.distance.distance(p1, p2).km)
print("OSRM", database.get_route_distance_km(int(id1), int(id2)))
